In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModel
import torch
import os

# 加载文本文件
input_file_path = "a.txt"
with open(input_file_path, "r", encoding="utf-8") as file:
    text = file.read()

# 拆分文本
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # 每个块的字符数
    chunk_overlap=50,  # 每个块的重叠字符数（用于上下文连接）
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
chunks = splitter.split_text(text)

# 加载本地模型
model_name = "/home/mhlee/Work/dev/vraft/src/python/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


/home/mhlee/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# 打开文件进行写入
output_file_path = "/tmp/text384.txt"
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

with open(output_file_path, "w", encoding="utf-8") as f:
    for i, chunk in enumerate(chunks):
        vector = embed_text(chunk)
        vector_str = ','.join(f"{x:.6f}" for x in vector)  # 格式化向量数据以提高可读性
        f.write(f"{i}; {vector_str}; {chunk}\n")
"""
        f.write(f"行号: {i}\n")
        f.write(f"向量数据: {vector_str}\n")
        f.write(f"分割的文本块: {chunk}\n")
        f.write("-" * 50 + "\n")
"""
print(f"数据已保存到 {output_file_path}")

数据已保存到 /tmp/text384.txt


In [ ]:
# create table --name=text_table --partition_num=1 --replica_num=1 --dim=384
# load --table=text_table --file=/tmp/text384.txt
# build index --table=text_table --annoy_tree_num=10
# getknn --table=text_table --key=2 --limit=20
# quit

In [3]:
qvector = embed_text("TDengine一个cpu可以处理几个vnode？")
print(qvector)



[-3.36260311e-02  4.31706905e-02 -2.88739890e-01 -3.01715910e-01
  5.28401919e-02 -3.01823139e-01  2.87331551e-01  2.45747492e-01
 -3.41862962e-02 -5.78543022e-02  3.64801288e-01 -3.52113605e-01
 -3.16007167e-01 -4.18510884e-01 -1.27047256e-01 -9.86277238e-02
  1.57578245e-01  2.18069889e-02  1.63505003e-02 -1.63388625e-01
  1.84790075e-01  4.79817390e-06 -1.33694738e-01  9.87115055e-02
  1.84715688e-01  2.96508044e-01  1.69409126e-01  2.86948025e-01
  2.95707822e-01 -1.80631448e-02 -8.20261538e-02 -7.08827451e-02
 -9.59695503e-02  2.61973768e-01  1.19065575e-01  2.72333361e-02
 -3.81127775e-01 -5.02971053e-01 -2.88562417e-01  1.14855580e-01
 -2.85811096e-01 -3.85802656e-01  9.84286144e-03 -2.05881432e-01
  1.32640421e-01  3.64709526e-01 -1.67322204e-01 -1.53929461e-02
 -2.31798857e-01 -2.40716875e-01 -1.45791560e-01 -3.02598119e-01
  1.43955857e-01  5.11340611e-03 -1.00724600e-01  1.47084728e-01
  2.22529575e-01  7.65588358e-02 -5.37613966e-02  6.39491677e-02
  2.46150494e-01 -3.07752

In [4]:
import sys
import os

# 将 vdb_module.so 的目录添加到 sys.path
sys.path.append('/home/mhlee/Work/dev/vraft/output/libs')

import vdb_module

# 创建 VdbEngine 实例
engine = vdb_module.VdbEngine("/tmp/local_console")

table = "text_table"
#vec = [0.172457, 0.383009, 0.255386, 0.016210, 0.705780, 0.920516, 0.678624, 0.796226, 0.115947, 0.185988]  # 示例向量
limit = 5

res_code, results = engine.get_knn_by_vector(table, qvector, limit)

bg_data = ""

print("Response code:", res_code)
for result in results:
    print(result.to_print_string())
    bg_data += result.attach_value

del engine

print(f"背景知识: {bg_data}")

Response code: 0
{distance:0.704123, key:18, attach_value:需求TDengine用户对CPU的需求主要受以下3个因素影响：数据分片：在TDengine中，每个CPU核心可以服务1至2个vnode。假设一个集群配置了100个vgroup，并且采用三副本策略，那么建议该集群的CPU核心数量为150~300个，以实现最佳性能。数据写入：TDengine的单核每秒至少能处理10,000}
{distance:0.848784, key:40, attach_value:TDengine集群所需的内存容量、CPU核数、存储空间以及网络带宽。若数据副本数不是1，还需要将总需求量乘以副本数以得到实际所需资源。得益于TDengine出色的水平扩展能力，我们可以轻松计算出资源需求的总量。接下来，只须将这个总量除以单台物理机或虚拟机的资源量，便能大致确定需要购买多少台物理机或虚拟机来部署TDengine集群。网络端口要求下表列出了TDengine}
{distance:0.894126, key:22, attach_value:CPU使用率不超过50%，以确保系统的稳定性和性能。一旦CPU使用率超过这一阈值，就需要考虑增加新的节点或增加CPU核心数量，以提供更多的计算资源。存储需求相较于传统通用数据库，TDengine在数据压缩方面表现出色，拥有极高的压缩率。在大多数应用场景中，TDengine的压缩率通常不低于5倍。在某些特定情况下，压缩率甚至可以达到10}
{distance:0.929597, key:17, attach_value:RESTful/WebSocket连接方式来访问TDengine集群，而不采用taosc原生连接方式。在绝大多数情形下，RESTful/WebSocket连接方式均满足业务写入和查询要求，并且该连接方式不依赖于taosc，集群服务器升级与客户端连接方式完全解耦，使得服务器维护、升级更容易。CPU需求TDengine用户对CPU的需求主要受以下3个因素影响：数据分片：在}
{distance:0.931361, key:26, attach_value:keep参数来优化存储成本仍显不足。为此，TDengine进一步创新，推出了多级存储策略。此外，为了加速数据处理流程，TDengine特别支持

In [8]:
query = "TDengine一个cpu可以处理几个vnode？"
rag_query="根据如下内容：“" + bg_data + "” " + "回答问题： “" + query + "”"

In [ ]:
# update api_key

In [9]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="xx",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

In [12]:
print("\n原始query：")
print(query)

completion = client.chat.completions.create(
    model="qwen-max", # 更多模型请参见模型列表文档。
    messages = [{"role": "user", "content": query}]
)
print("\n大模型回答：")
print(completion.choices[0].message.content)


原始query：
TDengine一个cpu可以处理几个vnode？

大模型回答：
TDengine的性能和可扩展性设计使得单个CPU可以处理较多的vnode（虚拟节点）。然而，具体能处理多少个vnode，并没有一个固定的数值，因为它依赖于多个因素，包括但不限于：

1. **硬件规格**：CPU的性能（如核心数、主频）、内存大小、磁盘速度等都会影响到单个CPU可以有效管理的vnode数量。
2. **负载情况**：数据写入和查询的频率及复杂度直接影响到系统负载，从而影响单CPU支持的vnode数量。
3. **vnode配置**：每个vnode的数据量、存储配置以及处理的任务类型（例如大量写入还是复杂查询）也会影响其资源消耗。
4. **系统优化与配置**：包括TDengine的配置参数（如缓存大小、线程池设置等）以及操作系统的优化程度。

通常，为了达到最佳性能，建议通过实际测试来确定最合适的vnode数量。可以从小规模开始部署，逐步增加vnode数量，同时监控系统资源使用情况和性能指标，以找到最佳的配置点。

TDengine的设计原则之一是简化运维和自动优化，因此它具备一定的自我调整能力，但合理的手动调优依然能进一步提升系统效率。在高并发或大规模数据场景下，建议进行细致的性能测试和调优，以确保系统能够高效稳定运行。


In [13]:

print("\nrag query：")
print(rag_query)

completion = client.chat.completions.create(
    model="qwen-max", # 更多模型请参见模型列表文档。
    messages = [{"role": "user", "content": rag_query}]
)
print("\n大模型回答：")
print(completion.choices[0].message.content)


rag query：
根据如下内容：“需求TDengine用户对CPU的需求主要受以下3个因素影响：数据分片：在TDengine中，每个CPU核心可以服务1至2个vnode。假设一个集群配置了100个vgroup，并且采用三副本策略，那么建议该集群的CPU核心数量为150~300个，以实现最佳性能。数据写入：TDengine的单核每秒至少能处理10,000TDengine集群所需的内存容量、CPU核数、存储空间以及网络带宽。若数据副本数不是1，还需要将总需求量乘以副本数以得到实际所需资源。得益于TDengine出色的水平扩展能力，我们可以轻松计算出资源需求的总量。接下来，只须将这个总量除以单台物理机或虚拟机的资源量，便能大致确定需要购买多少台物理机或虚拟机来部署TDengine集群。网络端口要求下表列出了TDengineCPU使用率不超过50%，以确保系统的稳定性和性能。一旦CPU使用率超过这一阈值，就需要考虑增加新的节点或增加CPU核心数量，以提供更多的计算资源。存储需求相较于传统通用数据库，TDengine在数据压缩方面表现出色，拥有极高的压缩率。在大多数应用场景中，TDengine的压缩率通常不低于5倍。在某些特定情况下，压缩率甚至可以达到10RESTful/WebSocket连接方式来访问TDengine集群，而不采用taosc原生连接方式。在绝大多数情形下，RESTful/WebSocket连接方式均满足业务写入和查询要求，并且该连接方式不依赖于taosc，集群服务器升级与客户端连接方式完全解耦，使得服务器维护、升级更容易。CPU需求TDengine用户对CPU的需求主要受以下3个因素影响：数据分片：在keep参数来优化存储成本仍显不足。为此，TDengine进一步创新，推出了多级存储策略。此外，为了加速数据处理流程，TDengine特别支持配置多块硬盘，以实现数据的并发写入与读取。这种并行处理机制能够最大化利用多核CPU的处理能力和硬盘I/O带宽，大幅提高数据传输速度，完美应对高并发、大数据量的应用场景挑战。技巧如何估算TDengine压缩率用户可以利用性能测试工具” 回答问题： “TDengine一个cpu可以处理几个vnode？”

大模型回答：
在TDengine中，一个CPU核心可以服务1至2个vnode。
